### 공동주택 코드 매핑

In [249]:
# 라이브러리
import os
import chardet
import pandas as pd
import re 

p = re.compile('\(([^)]+)')  

def jibun_juso(addr):
    d = p.findall(addr) 
    if isinstance(d, list) and len(d) > 0:
        return d[0]
    return None

def doro_juso(addr):
    p = addr.find("(")
    juso = addr[0:p-1].rstrip()
    return juso

def part_back(obj):
    # 데이터에 null 있음    
    if isinstance(obj, str) and len(obj) > 0:
        part_front = obj.split()[0]
        part_back = obj.replace(part_front, "").lstrip()
        return part_back
    else:
        return None
    
def part_front(obj):
    part_front = obj[1].replace(obj[0], "").rstrip()
    return part_front

def no_blank(obj):
    # 데이터에 null 있음    
    if isinstance(obj, str) and len(obj) > 0:
        re_str = obj.replace(" ", "")
        return re_str
    else:
        return None

In [250]:
# 데이터 파일 위치
data_root = "D:\PythonProject\data-gatherer\molit_apt\data"
# 아파트 코드 목록: 외부(국토교통부)
file_ext = os.path.join(data_root, "공동주택_국토부_apt_info_20240411.csv")
# 아파트 코드 목록: 내부(고객지원시스템)
file_int = os.path.join(data_root, "공동주택_고객지원시스템_20240415.csv")

# 파일의 인코딩 확인
with open(file_ext, 'rb') as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding_ext = result['encoding']
    
with open(file_int, 'rb') as f:
    rawdata = f.read()
    result = chardet.detect(rawdata)
    encoding_int = result['encoding']

* df_int

In [251]:
# 파일 데이터 읽기
data_int = pd.read_csv(file_int, encoding=encoding_int, dtype=object)
df_int = data_int.copy()
int_cnt = df_int.shape[0]

In [252]:
""" 데이터 정리 """
# 1. 공동주택이 아닌 데이터 제외
idx = df_int[df_int['용도'] != '공동주택'].index
df_int.drop(idx, inplace=True)
# 2. 필요한 항목만 남김
df_int = df_int[['아파트명', '아파트코드', '우편번호', '주소']]
# 3. 중복 데이터 없애기
df_int.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)
# 4. 시군구 항목 생성
df_int['시군구'] = df_int['주소'].apply(lambda x: x.split()[0])
# 5. 대구광역시 공급권역 데이터만 남기고 나머지(경북 지역) 데이터 삭제
idx = df_int[~df_int['시군구'].isin(['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군'])].index
df_int.drop(idx, inplace=True)
# 6. 지번주소 항목 분리 생성
df_int['지번주소'] = df_int['주소'].apply(jibun_juso)
df_int['지번주소_공백X'] = df_int['지번주소'].apply(no_blank)
# 7. 도로명주소 항목 분리 생성
df_int['시군구_도로명주소'] = df_int['주소'].apply(doro_juso)
df_int['도로명주소'] = df_int['시군구_도로명주소'].apply(part_back)
df_int['도로명주소_공백X'] = df_int['도로명주소'].apply(no_blank)
# 8. 아파트명 공백X
df_int['아파트명_공백X'] = df_int['아파트명'].apply(lambda x: x.replace(" ",""))
# 9. 필요항 컬럼만 구성
df_int = df_int[['아파트코드','아파트명', '아파트명_공백X', '시군구', '지번주소', '지번주소_공백X', '도로명주소', '도로명주소_공백X']]

* df_ext

In [255]:
# 파일 데이터 읽기
data_ext = pd.read_csv(file_ext, encoding=encoding_ext, dtype=object)
df_ext = data_ext.copy()
ext_cnt = df_ext.shape[0]

In [256]:
# df_ext.isnull().sum()

In [257]:
""" 데이터 정리 """
# 1. 필요한 항목만 남김
df_ext = df_ext[['kapt_cd', 'kapt_nm', 'as2', 'as3', 'as4', 'kapt_addr', 'doro_juso']]
# 2. 시군구_도로명주소 항목 생성
df_ext['시군구_도로명주소'] = df_ext['doro_juso'].str.lstrip('대구광역시')
# 3. 도로명주소 항목 생성
df_ext['도로명주소'] = df_ext['시군구_도로명주소'].apply(part_back)
df_ext['도로명주소_공백X'] = df_ext['도로명주소'].apply(no_blank)
# 4. 법정동주소 항목 생성   
df_ext['광역시_법정동주소'] = df_ext[['kapt_nm', 'kapt_addr']].apply(part_front, axis=1)
df_ext['시군구_법정동주소'] = df_ext['광역시_법정동주소'].apply(part_back)
df_ext['법정동주소'] = df_ext['시군구_법정동주소'].apply(part_back)
df_ext['법정동주소_공백X'] = df_ext['법정동주소'].apply(no_blank)
# 5. 컬럼명 변경
df_ext.rename(columns={'kapt_nm' : '아파트명', 'as2' : '시군구'}, inplace=True)
# 6. 아파트명 공백X
df_ext['아파트명_공백X'] = df_ext['아파트명'].apply(lambda x: x.replace(" ",""))
# 7. 필요한 컬럼만 구성
df_ext = df_ext[['kapt_cd', '아파트명', '아파트명_공백X', '시군구', 'as3', 'as4', '도로명주소', '도로명주소_공백X', '법정동주소', '법정동주소_공백X']]

* 1차 매칭 - 도로명주소

In [258]:
# 1차 매칭
df_merge1 = pd.merge(df_ext, df_int, how='outer', left_on="도로명주소_공백X", right_on="도로명주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match1 = df_merge1[df_merge1['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match1.drop_duplicates(subset='kapt_cd', inplace=True)       # N:1 삭제 (abnormal mapping)
df_match1.drop_duplicates(subset='아파트코드', inplace=True)     # 1:N 삭제 (abnormal mapping)
df_match1['match'] = '1. 도로명주소 일치'
# 1차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match1['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match1['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0]

* 2차 매칭 - 아파트명

In [259]:
# 2차 매칭
df_merge2 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X", right_on="아파트명_공백X", suffixes=('_left','_right'), indicator=True)
df_match2 = df_merge2[df_merge2['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match2.drop_duplicates(subset='kapt_cd', inplace=True)       # N:1 삭제 (abnormal mapping)
df_match2.drop_duplicates(subset='아파트코드', inplace=True)     # 1:N 삭제 (abnormal mapping)
df_match2['match'] = '2. 아파트명 일치'
# 2차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match2['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match2['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0]

* 3차 매칭 - [EXT]법정동주소 == [INT]지번주소

In [260]:
# 3차 매칭
df_merge3 = pd.merge(df_ext, df_int, how='outer', left_on="법정동주소_공백X", right_on="지번주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match3 = df_merge3[df_merge3['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match3.drop_duplicates(subset='kapt_cd', inplace=True)       # N:1 삭제 (abnormal mapping)
df_match3.drop_duplicates(subset='아파트코드', inplace=True)     # 1:N 삭제 (abnormal mapping)
df_match3['match'] = '3. 국토부_법정동주소와 내부시스템_지번주소 일치'
# 3차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match3['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match3['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0]

* 4차 매칭

In [261]:
# 4차 매칭
df_ext['as3_도로명주소_공백X'] = df_ext['as3'] + df_ext['도로명주소_공백X']
df_merge4 = pd.merge(df_ext, df_int, how='outer', left_on="as3_도로명주소_공백X", right_on="도로명주소_공백X", suffixes=('_left','_right'), indicator=True)
df_match4 = df_merge4[df_merge4['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match4.drop_duplicates(subset='kapt_cd', inplace=True)       # N:1 삭제 (abnormal mapping)
df_match4.drop_duplicates(subset='아파트코드', inplace=True)     # 1:N 삭제 (abnormal mapping)
df_match4['match'] = '4. 국토부_읍면+도로명주소와 내부시스템_도로명주소 일치'
# 4차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match4['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match4['아파트코드'])].index
df_int.drop(idx_int, inplace=True)
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0] - df_match4.shape[0]

* 5차 매칭

In [262]:
same_char = {"LH" : "엘에이치", "e" : "이"}
def re_char(apt_name):
    y = apt_name
    for k, v in same_char.items():
        if k in apt_name:
            y = apt_name.replace(k, v)
    return y

# 5차 매칭
df_ext['아파트명_공백X_단어통일'] = df_ext['아파트명_공백X'].apply(re_char)
df_int['아파트명_공백X_단어통일'] = df_int['아파트명_공백X'].apply(re_char)
df_merge5 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X_단어통일", right_on="아파트명_공백X_단어통일", suffixes=('_left','_right'), indicator=True)
df_match5 = df_merge5[df_merge5['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match5.drop_duplicates(subset='kapt_cd', inplace=True)       # N:1 삭제 (abnormal mapping)
df_match5.drop_duplicates(subset='아파트코드', inplace=True)     # 1:N 삭제 (abnormal mapping)
df_match5['match'] = '5. 아파트명 유사단어 통일 후 일치'
# 5차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match5['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match5['아파트코드'])].index
df_int.drop(idx_int, inplace=True)   
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0] - df_match4.shape[0] - df_match5.shape[0]

* 6차 매칭

In [263]:
extra_char = ['대구', '아파트']
def del_char(apt_name):
    y = apt_name
    for c in extra_char:
        if c in apt_name:
            y = apt_name.replace(c, "")     # 해당 단어 삭제
    return y

# 6차 매칭
df_ext['아파트명_공백X_부가단어삭제'] = df_ext['아파트명_공백X'].apply(del_char)
df_int['아파트명_공백X_부가단어삭제'] = df_int['아파트명_공백X'].apply(del_char)
df_merge6 = pd.merge(df_ext, df_int, how='outer', left_on="아파트명_공백X_부가단어삭제", right_on="아파트명_공백X_부가단어삭제", suffixes=('_left','_right'), indicator=True)
df_match6 = df_merge6[df_merge6['_merge'] == 'both'][['kapt_cd', '아파트코드']]
df_match6.drop_duplicates(subset='kapt_cd', inplace=True)       # N:1 삭제 (abnormal mapping)
df_match6.drop_duplicates(subset='아파트코드', inplace=True)     # 1:N 삭제 (abnormal mapping)
df_match6['match'] = '6. 아파트명 부가단어 삭제 후 일치'
# 6차 매칭된 데이터 삭제
idx_ext = df_ext[df_ext['kapt_cd'].isin(df_match6['kapt_cd'])].index
df_ext.drop(idx_ext, inplace=True)
idx_int = df_int[df_int['아파트코드'].isin(df_match6['아파트코드'])].index
df_int.drop(idx_int, inplace=True)   
assert df_ext.shape[0] == ext_cnt - df_match1.shape[0] - df_match2.shape[0] - df_match3.shape[0] - df_match4.shape[0] - df_match5.shape[0] - df_match6.shape[0]

* 총합

In [264]:
m1 = df_match1.shape[0]
m2 = df_match2.shape[0]
m3 = df_match3.shape[0]
m4 = df_match4.shape[0]
m5 = df_match5.shape[0]
m6 = df_match6.shape[0]
total = m1 + m2 + m3 + m4 + m5 + m6
print(f"대구광역시 국토부 데이터:\t {ext_cnt}")
print(f"고객지원시스템 데이터:\t {int_cnt}")
print(f"매칭: 1차 매칭 {m1}, 2차 매칭 {m2}, 3차 매칭 {m3}, 4차 매칭 {m4}, 5차 매칭 {m5}, 6차 매칭 {m6}")
print(f"총 매칭 {total}, 잔여 건수(국토부 데이터 기준) {ext_cnt - total}")

대구광역시 국토부 데이터:	 1039
고객지원시스템 데이터:	 3170
매칭: 1차 매칭 845, 2차 매칭 107, 3차 매칭 30, 4차 매칭 42, 5차 매칭 4, 6차 매칭 4
총 매칭 1032, 잔여 건수(국토부 데이터 기준) 7


In [265]:
# 매칭 데이터 병합
df_list = [df_match1, df_match2, df_match3, df_match4, df_match5, df_match6]
df_all = pd.concat(df_list, ignore_index=True)
df_all.sort_values(by='match')['match'].value_counts()

match
1. 도로명주소 일치                        845
2. 아파트명 일치                         107
4. 국토부_읍면+도로명주소와 내부시스템_도로명주소 일치     42
3. 국토부_법정동주소와 내부시스템_지번주소 일치         30
5. 아파트명 유사단어 통일 후 일치                 4
6. 아파트명 부가단어 삭제 후 일치                 4
Name: count, dtype: int64

In [266]:
df_all.shape[0]

1032

In [ ]:
# file_ext_unmatched = os.path.join(data_root, "공동주택_국토부_apt_info_20240411_unmatched.xlsx")
# df_ext.to_excel(file_ext_unmatched, index=False)

* 원 데이터 검색

In [ ]:
data_ext.columns

Index(['sido_cd', 'kapt_cd', 'kapt_nm', 'as1', 'as2', 'as3', 'as4', 'stdg_cd',
       'kapt_addr', 'cd_sale_nm', 'cd_heat_nm', 'kapt_tarea', 'kapt_dong_cnt',
       'kapt_da_cnt', 'kapt_cnst_co', 'kapt_dev_co', 'kapt_tel', 'kapt_fax',
       'kapt_url', 'cd_apt_nm', 'doro_juso', 'ho_cnt', 'cd_mgr_nm',
       'cd_hall_nm', 'kapt_use_dt', 'kapt_marea', 'kapt_mparea_60',
       'kapt_mparea_85', 'kapt_mparea_135', 'kapt_mparea_136', 'priv_area',
       'cr_dt', 'cr_id'],
      dtype='object')

In [ ]:
data_ext[data_ext['kapt_nm'].str.contains('서대구역')][['kapt_nm', 'kapt_addr','as1','as2','as3','as4','doro_juso']]

,kapt_nm,kapt_addr,as1,as2,as3,as4,doro_juso
12,서대구역화성파크드림,대구광역시 서구 평리동 1512-10 서대구역화성파크드림,대구광역시,서구,평리동,NaN,대구광역시 서구 서대구로29길 30
20,서대구역반도유보라센텀,대구광역시 서구 평리동 1083-2 서대구역반도유보라센텀,대구광역시,서구,평리동,NaN,대구광역시 서구 문화로 230
46,서대구역서한이다음 더 퍼스트,대구광역시 서구 평리동 1481-10 서대구역서한이다음 더 퍼스트,대구광역시,서구,평리동,NaN,NaN


In [ ]:
data_int[data_int['아파트명'].str.contains('가람')]

,아파트명,아파트코드,세대수,우편번호,주소,난방구분,용도,공동주택계약,설치위치,서비스센터,Unnamed: 10
292,가람타운,010229,559,41203,동구 신암로16길 16 (신천동 551-23),개별난방,공동주택,아니오,베란다,제5서비스(주),NaN
337,가람빌라,010504,8,41153,동구 해동로 135-1 (입석동 938-5),개별난방,공동주택,아니오,베란다,제5서비스(주),NaN
451,가람타운,200796,558,41203,동구 신암로16길 33 (신천동 551-35),개별난방,공동주택,아니오,베란다,제5서비스(주),NaN
950,가람마을,043804,919,42837,달서구 한실로 135 (도원동 1439),지역난방,공동주택,아니오,베란다,서비스2센터(주)월배,NaN
1331,가람마을,043804,526,42837,달서구 한실로 135 (도원동 1439),지역난방,공동주택,아니오,복도,서비스2센터(주)월배,NaN
1873,가람빌라,200653,6,41153,동구 해동로 135 (입석동 938-11),개별난방,공동주택,아니오,베란다,제5서비스(주),NaN
1886,가람빌라,200352,8,42693,달서구 와룡로15길 52 (본리동 430-6),개별난방,공동주택,아니오,베란다,서비스3센터(주),NaN
2005,가람타운,200795,259,41203,동구 신암로16길 17 (신천동 551-31),개별난방,공동주택,아니오,베란다,제5서비스(주),NaN
2321,가람빌라,041218,8,42693,달서구 구마로11길 47 (본리동 430-3),개별난방,공동주택,아니오,베란다,서비스3센터(주),NaN
2541,우방한가람타운,060316,207,42130,수성구 명덕로 368 (수성동1가 272-11),개별난방,공동주택,아니오,복도,제6서비스(주),NaN
